# Exploratória / Distribuições

In [7]:
import pandas as pd
import sweetviz as sv

# Carregar o dataset (substitua pelo seu próprio dataset)
# Exemplo: carregando um dataset de churn
url = '../env/dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv'
data = pd.read_csv(url)

data = data.drop('tenure', axis='columns')
data = data.drop('MonthlyCharges', axis='columns')

# Gerar o relatório de análise do dataset com Sweetviz
report = sv.analyze(data)

# Gerar o arquivo HTML com o relatório
report.show_html('telco_customer_churn_report.html')


Feature: MonthlyCharges                      |███████████████████████████████████▏     | [ 86%]   00:19 -> (00:03 left)
Done! Use 'show' commands to display/save.   |█████████████████████████████████████████| [100%]   00:01 -> (00:00 left)


Report telco_customer_churn_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


# 1. Importação das Bibliotecas Necessárias

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Para dividir os dados e avaliar o modelo
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve

# Algoritmos de Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# Ignorar avisos
import warnings
warnings.filterwarnings('ignore')

# 2. Carregamento e Exploração Inicial dos Dados

In [ ]:
# Carregar o dataset
data = pd.read_csv('../env/dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Visualizar as primeiras linhas
print("Amostra dos dados:")
display(data.head())

# Informações sobre o dataset
print("\nInformações sobre o dataset:")
data.info()

# Verificar valores nulos
print("\nValores nulos por coluna:")
print(data.isnull().sum())

# 3. Pré-processamento dos Dados

In [ ]:
# 3.1. Tratamento de valores ausentes
# 'TotalCharges' está como objeto, precisamos converter para numérico
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

In [ ]:
# Remover linhas com 'TotalCharges' nulo
data.dropna(subset=['TotalCharges'], inplace=True)

In [ ]:
# 3.2. Remover colunas irrelevantes
data.drop(['customerID'], axis=1, inplace=True)

In [ ]:
# 3.3. Separar variáveis por tipo
# Variáveis numéricas
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [ ]:
# Variável alvo
target_col = 'Churn'

# Variáveis categóricas (excluindo a variável alvo)
categorical_cols = data.drop(columns=numeric_cols + [target_col]).columns.tolist()

In [ ]:
categorical_cols

In [ ]:
# 3.4. Dividir X e y
X = data.drop(target_col, axis=1)
y = data[target_col].map({'Yes': 1, 'No': 0})  # Mapear a variável alvo para 0 e 1

In [ ]:
# 3.5. Pré-processamento usando Pipeline
# Pipeline para as variáveis categóricas
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [ ]:
# Pipeline para as variáveis numéricas
numeric_transformer = StandardScaler()

In [ ]:
# Combinar os transformadores usando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Criar o Pipeline completo
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# 4. Divisão dos Dados em Conjunto de Treino e Teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

In [ ]:
print(f"\nTamanho do conjunto de treinamento: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")

# 5. Treinamento e Avaliação dos Modelos

In [ ]:
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"\n{model_name} Metrics:")
    print(f"Acurácia: {acc:.4f}")
    print(f"Precisão: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-Score: {f1:.4f}")
    
    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Matriz de Confusão - {model_name}')
    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.show()
    
    # Curva ROC e AUC
    y_pred_proba = model.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr, tpr, label=f"AUC = {auc:.4f}")
    plt.plot([0,1], [0,1], linestyle='--')
    plt.title(f'Curva ROC - {model_name}')
    plt.xlabel('Taxa de Falsos Positivos')
    plt.ylabel('Taxa de Verdadeiros Positivos')
    plt.legend()
    plt.show()

### 5.1. Regressão Logística

In [ ]:

clf_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

In [ ]:
clf_lr.fit(X_train, y_train)
evaluate_model(clf_lr, X_test, y_test, "Regressão Logística")

### 5.2. Random Forest

In [ ]:

clf_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [ ]:
clf_rf.fit(X_train, y_train)
evaluate_model(clf_rf, X_test, y_test, "Random Forest")

### 5.3. XGBoost

In [ ]:

clf_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
])

clf_xgb.fit(X_train, y_train)
evaluate_model(clf_xgb, X_test, y_test, "XGBoost")

In [ ]:
# 6. Otimização de Hiperparâmetros para Random Forest

param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(estimator=clf_rf, param_grid=param_grid, cv=3, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

In [ ]:
print(f"\nMelhores parâmetros para Random Forest: {grid_search.best_params_}")
best_rf = grid_search.best_estimator_

# Avaliar modelo otimizado
evaluate_model(best_rf, X_test, y_test, "Random Forest Otimizado")

In [ ]:


# 7. Conclusões e Próximos Passos
print("""
Conclusões:
- Identificamos que o modelo Random Forest Otimizado apresenta o melhor desempenho em termos de F1-Score.
- As variáveis mais importantes para a previsão de churn podem ser investigadas a partir do modelo treinado.

Próximos Passos:
- Implementar técnicas de balanceamento de classes se necessário (e.g., SMOTE).
- Explorar modelos adicionais ou ensemble de modelos.
- Implementar o modelo em produção utilizando AWS SageMaker.
- Monitorar o desempenho do modelo e atualizá-lo periodicamente.
""")
